In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm 
tqdm.pandas()

In [ ]:
# 카테고리화 사용자 함수들

def cat_week(x):
    cat = 0
    if x[:2]=='주중': pass
    else: cat+= 4
        
    if x[-2:]=='새벽': pass
    elif x[-2:]=='오전': cat+=1
    elif x[-2:]=='오후': cat+=2
    else: cat+=3
        
    return cat


def cat_item(x):
    if x == '가공식품':
        return 1
    elif x == '가구/인테리어':
        return 2
    elif x == '가전':
        return 3
    elif x == '교육/문화용품':
        return 4
    elif x == '기타':
        return 5
    elif x == '명품':
        return 6
    elif x == '신선식품':
        return 7
    elif x == '외식':
        return 8
    elif x == '의류':
        return 9
    elif x == '일상용품':
        return 10
    elif x == '전문스포츠/레저':
        return 11
    elif x == '패션':
        return 12
    else:
        return 0
    
def cat_level(x):
    if x == '없음':
        return 0
    elif x == '일반':
        return 1
    elif x == '우수':
        return 2
    elif x== '최우수':
        return 3
    else:
        return 4


def cat_channel(x):
    if x == 'A':
        return 0
    elif x == 'B':
        return 1
    elif x== 'C':
        return 2
    else:
        return 3

    
# 등급 사용자 함수
def cust_level(x):
    if x >= 34350000: return 'VIP'
    elif x >= 20150000: return '최우수'
    elif x >= 8700000: return '우수'
    elif x == 0: return '없음'
    else: return '일반'

    
# def mart_level(x):
#     if x >= 3000000: return 'VIP'
#     elif x >= 1800000: return '최우수'
#     elif x >= 900000: return '우수'
#     elif x == 0: return '없음'
#     else: return '일반'
    
# def super_level(x):
#     if x >= 1200000: return 'VIP'
#     elif x >= 700000: return '최우수'
#     elif x >= 300000: return '우수'
#     elif x == 0: return '없음'
#     else: return '일반'
    
# def dep_level(x):
#     if x >= 30000000: return 'VIP'
#     elif x >= 17500000: return '최우수'
#     elif x >= 7500000: return '우수'
#     elif x == 0: return '없음'
#     else: return '일반'




In [ ]:
# 총 구매액 함수
def amount(df):
    df1 = df.groupby('cust_no')[['pur_amount']].sum().reset_index()
    df1.columns=['cust_no','amount']
    return df1

# 반기 데이터 정제 함수들

ratio = lambda x : x/x.sum()
    

    
################################ 최다 구매 시기 (week) 함수 ######################################

def dtime(df):
    origin_df = df
    
    
    # 최다 구매 시기
    
#     df1 = origin_df.pivot_table(index='cust_no',columns='pur_week_time',values='half',aggfunc='count')
#     # df1 = 요일별 count 값이 들어간 데이터프레임
    
#     df1['week'] = df1.idxmax(axis=1)
#     # count값이 가장 큰 column 이름반환 (column name == 요일 category)
    
#     df1.reset_index(inplace=True)
#     df1 = df1[['cust_no','week']]
    
    #------------------------level 코드 삭제 -------------------------------------*
    
    ser = pd.DataFrame(origin_df.groupby('cust_no')['pur_amount'].sum()).reset_index()
    ser.columns=['cust_no','level']
#     df1 = pd.merge(df1,ser,on='cust_no',how='outer')
    
    # level/방문요일 카테고리화
#     ser['level'] = ser['level'].apply(cust_level)
#     ser['level'] = ser['level'].apply(cat_level)

    #---------------------------------------------------------------------------*
    
#     # 요일column 함수 사용하여 labeling 
#     df1['week'] = df1['week'].apply(cat_week)
    
    return ser
    
    
    
################################ 최다 이용 제휴사 함수 삭제  ####################################  

# def most_channel(df):
#     origin_df = df
    
#     # 최다구매제휴사
#     df1 =origin_df.pivot_table(index='cust_no',columns='channel', values='pur_amount', aggfunc='count')
#     df1['most_channel'] = df1.idxmax(axis=1)
#     df1.reset_index(inplace=True)
#     df1 = df1[['cust_no','most_channel']]
    
#     # 제휴사 카테고리화
#     df1['most_channel'] = df1['most_channel'].apply(cat_channel)

#     return df1



################################### 구매 아이템품목 count 함수 ###################################


def item_cnt(df):
    # 구매 아이템 품목
    origin_df = df.copy()

    df2 =origin_df.pivot_table(index='cust_no',columns='cat_dae',values='pur_amount',aggfunc='count')
    df2['item_cnt'] = df2.apply(lambda x : x.notna().sum(), axis=1)
    df2.reset_index(inplace=True)
    df1 = df2[['cust_no','item_cnt']]
    return df1


################################# 아이템 구매 관련 함수 #########################################
    
def cnt_item_prob(df):
    origin_df = df.copy()
    
    df2 =origin_df.pivot_table(index='cust_no',columns='cat_dae',values='pur_amount',aggfunc=['sum','count'])
    # df2 = 대분류에 따라 구매액을 count, sum 한 데이터 프레임
    
    #------------------ recency 구하는 식 삭제 ---------------------------#
    
    origin_df['pur_date']=pd.to_datetime(origin_df['pur_date'])
    sample_dt = origin_df.loc[origin_df.index[0],'pur_date']
    if sample_dt.month < 7:
        dt = datetime(sample_dt.year, 7, 1)
    else:
        dt = datetime(sample_dt.year+1, 1, 1)
    
    
    # 가장 최근 방문일
    df1 = origin_df.groupby('cust_no')['pur_date'].max().reset_index()
    df1.columns=['cust_no','recency']
    df1['recency'] = df1['recency'].apply(lambda x : (dt-x).days )
    
   #--------------------------------------------------------------------#

    
    # 제휴사 구매횟수
    ser = df2['count'].sum(axis=1).reset_index()
    ser.columns = ['cust_no','channel_freq']
    df1 = pd.merge(df1,ser,on='cust_no',how='outer')
    
    # 제휴사 구매품목개수
    ser = item_cnt(origin_df)
    df1 = pd.merge(df1,ser,on='cust_no',how='outer')
    
    # 제휴사 최고구매액품목
    ser = df2['sum'].idxmax(axis=1).reset_index()
    ser.columns=['cust_no','max_item']
    df1 = pd.merge(df1,ser,on='cust_no',how='outer')
    
    
    
    # 제휴사 최고 구매액 품목 비중
    ser = df2['sum'].fillna(0).apply(lambda x : x.max()/x.sum(), axis=1).reset_index()
    ser.columns = ['cust_no','max_item_ratio']
    df1 = pd.merge(df1,ser,on='cust_no',how='outer')
   
    #---------------- 불평등지수 구하는 식 삭제 ---------------------#
    
#     ser = df2['sum'].fillna(0).apply(lambda x : x/x.sum(), axis=1)
#     ser['gini'] = ser.apply(lambda x : np.sum(np.power(x,2)),axis=1)
#     ser.reset_index(inplace=True)
#     ser = ser[['cust_no','gini']]
#     df1 = pd.merge(df1,ser,on='cust_no',how='outer')

    #--------------------------------------------------------------#
    
    # 제휴사 최고 구매액 품목 구매 건수
    df1['max_item_cnt'] = df1[['cust_no','max_item']].apply(lambda x: df2['count'][x[1]][x[0]], axis=1)
    
    
    # 중분류 카테고리화
    df1['max_item'] = df1['max_item'].apply(cat_item)
    
    # 구매품목 비중
    
    ser = df2['sum'].fillna(0).apply(lambda x : x/x.sum(), axis=1).reset_index()
    
    df1 = pd.merge(df1,ser,on='cust_no',how='outer')
    
    ser = df2['sum'].fillna(0).reset_index()
    
    df1 = pd.merge(df1,ser,on='cust_no',how='outer',suffixes=['','_sum'])
    
    
    
    return df1


################################### 최종 df 복합 처리 함수 ################################

def last_(df):
    df1 = df
    
    df2 = dtime(df1)
    # 시간 정보 추가
    
    df3 = amount(df1)
#     총 구매액 함수
    
    df2 = pd.merge(df2, df3, on='cust_no',how='outer')
    # 위의 데이터 프레임 merge
    
    df3 = item_cnt(df1)
    # 총 아이템 개수 
    
    df2 = pd.merge(df2, df3, on='cust_no',how='outer')
    
    
    # 제휴사별 데이터
    for i in ['A','B','C']:
        tmp_df = df1.query(f'channel=="{i}"')
        df3 = cnt_item_prob(tmp_df)        # 아이템 구매 함수
        
        col = df3.columns
        col = [x+'_'+i for x in col]     # column name에 제휴사 이름 추가
        col[0] = 'cust_no'
        df3.columns=col
        
        df2 = pd.merge(df2, df3, on='cust_no',how='outer')
        
        
        
        df3 = amount(tmp_df)       # 총 구매액 함수
        col = df3.columns
        col = [x+'_'+i for x in col]     # column name에 제휴사 이름 추가
        col[0] = 'cust_no'
        df3.columns=col
        
        df2 = pd.merge(df2, df3, on='cust_no',how='outer')
    
    df2.fillna(0,inplace=True)    # nan값 0처리
    df2['level'] = df2['level'].apply(cust_level)

    
    df2['level'] = df2['level'].apply(cat_level)
    
    #------------- gini계수 관련 삭제 ------------------#
#     for i in ['A','B','C']:
#         df2[f'gini_{i}'] = 1-df2[f'gini_{i}']
        
    #--------------------------------------------------#
        
    return df2



In [ ]:
def merge_df(df,lb1=1, lb2=2):
    origin_df = df.fillna(0)
    del_df = origin_df.copy()

    for i in ['channel_freq','item_cnt','max_item_ratio','max_item_cnt']:
        for j in ['_A','_B','_C']:
            
            origin_df[f'{i}{j}_ch'] = origin_df[f'{i}{j}_{lb2}'] - origin_df[f'{i}{j}_{lb1}']
            del_df[f'{i}{j}_ch'] = origin_df[f'{i}{j}_{lb2}'] - origin_df[f'{i}{j}_{lb1}']
            del_df.drop([f'{i}{j}_{lb1}',f'{i}{j}_{lb2}'],axis=1,inplace=True)
    
    return del_df, origin_df

In [ ]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA

# def pca_df(df,columns,lb1=1, lb2=2):
#     origin_df = df.fillna(0)
#     del_df = origin_df.copy()

#     for i in columns:
# #         for j in ['_A','_B','_C']:
#         scaler = StandardScaler()
#         pca = PCA(n_components=1)

#         origin_df[f'{i}_{lb2}'] = scaler.fit_transform(origin_df[[f'{i}_{lb2}']])
#         origin_df[f'{i}_{lb1}'] = scaler.fit_transform(origin_df[[f'{i}_{lb1}']])


#         origin_df[f'{i}_pca'] = pca.fit_transform(origin_df[[f'{i}_{lb2}',f'{i}_{lb1}']])
#         del_df[f'{i}_pca'] =pca.fit_transform(origin_df[[f'{i}_{lb2}',f'{i}_{lb1}']])
#         del_df.drop([f'{i}_{lb1}',f'{i}_{lb2}'],axis=1,inplace=True)
    
#     return del_df, origin_df

In [ ]:
def weight(df,col,new_name):
    origin_df = df.copy()
    qcut_1 = pd.qcut(origin_df.loc[origin_df[col]>0,col],[0,0.2,0.4,0.6,0.8,1]).unique()
    qcut_2 = pd.qcut(origin_df.loc[origin_df[col]<0,col],[0,0.2,0.4,0.6,0.8,1]).unique()

    origin_df.loc[origin_df[col]==0,new_name] = origin_df.loc[origin_df[col]==0,col].apply(lambda x : 0*rank(x, qcut_1))
    
    origin_df.loc[origin_df[col]>0,new_name] = origin_df.loc[origin_df[col]>0,col].progress_apply(lambda x : 0.1*rank(x, qcut_1))
    origin_df.loc[origin_df[col]<0,new_name] = origin_df.loc[origin_df[col]<0,col].progress_apply(lambda x : -0.1*rank(x, qcut_2,False))
    origin_df[new_name].fillna(0,inplace=True)
    return origin_df

In [ ]:

def qcut(series,cut=[0,0.04, 0.11, 0.23,0.4,0.6,0.77,0.89,0.96,1]):
    qcut = pd.qcut(series.loc[series != 0],cut).unique()
    return qcut


def rank(x, qcut,ascending = True):
    qcut = qcut.sort_values(ascending=ascending)
    if x == 0:
        return 0
        
    for idx,cut in enumerate(qcut):
        if x in cut:
            return idx +1 
        
# del_df['cut_amount_1'] = del_df['amount_1'].progress_apply(lambda x : rank(x, qcut(del_df['amount_1'])))
# del_df['cut_amount_2'] = del_df['amount_2'].progress_apply(lambda x : rank(x, qcut(del_df['amount_2'])))